In [ ]:
# !pip install openai, moviepy, pytube, pydub, ffmpeg-python

## YouTube 에서 다운로드 후 음성 추출


In [ ]:
from pytube import YouTube
import os

# 유튜브 링크
link = "https://youtu.be/LQS3y7Tckhc?si=pEJdAfSq3On_Uvou"

yt = YouTube(link)
filename = yt.streams.filter(only_audio=True).first().download()
renamed_file = filename.replace(".mp4", ".mp3")
os.rename(filename, renamed_file)

In [ ]:
# 다운로드 받은 유튜브 음성 파일을 mp3 파일로 변환
# !ffmpeg -i "[GPTs로 꿀빨기] PPT 제작 자동화.mp3" -vn -ar 44100 -ac 2 -b:a 192k youtube_audio.wav -y

In [ ]:
from IPython.display import Audio

Audio("youtube_audio.wav")

## 로컬 비디오 파일을 오디오 파일로 변환하기


In [ ]:
import moviepy.editor

# 로컬 파일에서 음성 추출
original_file = "Wispher로-자막작업하기_exported.mp4"
renamed_file = original_file[:-4] + ".mp3"
video = moviepy.editor.VideoFileClip(original_file)
video.audio.write_audiofile(renamed_file)

In [ ]:
from IPython.display import Audio

Audio(renamed_file)

## Whisper


In [ ]:
## 아래는 whisper 토큰 정보를 위한 참고 링크
## http://www.teddynote.com/python/chatgpt-python-api/ 

In [ ]:
# 토큰 정보로드를 위한 라이브러리
# 설치: pip install python-dotenv
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

In [ ]:
from openai import OpenAI

client = OpenAI()

In [ ]:
# 전체 음성파일에 대한 트래스크립트 생성
audio_file = open(renamed_file, "rb")
transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    language="ko",
    response_format="text",
    temperature=0.0,
)

print(transcript)

## Wav 파일로 저장


In [ ]:
from pydub import AudioSegment

# files
src = renamed_file
dst = renamed_file[:-4] + ".wav"

# convert wav to mp3
audSeg = AudioSegment.from_mp3(src)
audSeg.export(dst, format="wav")

In [ ]:
target_audio = dst
target_audio

## Audio Segment


In [1]:
from pydub import AudioSegment
from pydub.silence import split_on_silence, detect_silence

# 오디오 파일 불러오기
audio = AudioSegment.from_file(target_audio, format="wav")

min_silence_len = 350  # 무음으로 간주될 최소 길이 (밀리초 단위)
silence_thresh = -35  # 무음으로 간주될 데시벨 값

# 무음 부분을 기준으로 오디오 분할
chunks = split_on_silence(
    audio,
    min_silence_len=min_silence_len,  # 무음으로 간주될 최소 길이 (밀리초 단위)
    silence_thresh=silence_thresh,  # 무음으로 간주될 데시벨 값
    keep_silence=0,
)
print(len(chunks))

silences = detect_silence(
    audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh
)
silences[:10]
silences_diff = [s[1] - s[0] for s in silences]

NameError: name 'target_audio' is not defined

In [ ]:
output = chunks[0]
output += audio[silences[0][0] : silences[0][1]]
for i in range(1, 11):
    output += chunks[i]
    output += audio[silences[i][0] : silences[i][1]]

output

In [ ]:
from tqdm import tqdm

# 분할된 각 청크를 파일로 저장 (예시)
current_duration = 0.0
timeline = []
transcripts = []

for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
    f_name = "sample/chunk{i}.wav"
    chunk.export(f_name, format="wav")
    try:
        transcript = client.audio.transcriptions.create(
            file=open(f_name, "rb"),
            model="whisper-1",
            language="ko",
            response_format="text",
            temperature=0.0,
        )
    except:
        transcript = ""

    start = current_duration
    if i < len(silences_diff):
        end = current_duration + chunk.duration_seconds * \
            1000 + silences_diff[i]
    else:
        end = current_duration + chunk.duration_seconds * 1000

    print(int(start), int(end))
    timeline.append((start, end))
    current_duration = end
    transcripts.append(transcript)

In [ ]:
def format_time(ms):
    """밀리초를 SRT 포맷의 시간 문자열로 변환합니다."""
    seconds, milliseconds = divmod(ms, 1000)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},{int(milliseconds):03d}"


def create_srt(chunks, subtitles, filename):
    """SRT 파일을 생성합니다."""
    with open(filename, "w") as file:
        combined = [
            (start, end, text) for (start, end), text in zip(timeline, transcripts)
        ]
        for i, (start, end, text) in enumerate(combined, start=1):
            file.write(f"{i}\n")
            file.write(f"{format_time(start)} --> {format_time(end)}\n")
            file.write(f"{text}\n\n")


# SRT 파일 생성
create_srt(timeline, transcripts, "subtitles.srt")